In [3]:
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import faiss
import numpy as np
import os

from pillow_heif import register_heif_opener

register_heif_opener()

In [ ]:
#Define a function that normalizes embeddings and add them to the index
def add_vector_to_index(embedding, index):
    #convert embedding to numpy
    vector = embedding.detach().cpu().numpy()
    #Convert to float32 numpy
    vector = np.float32(vector)
    #Normalize vector: important to avoid wrong results when searching
    faiss.normalize_L2(vector)
    #Add to index
    index.add(vector)


    #Populate the images variable with all the images in the dataset folder
images = []
for root, dirs, files in os.walk('../gallery/photos/'):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.heic'):
            images.append(root + '/' + file)

print('Number of images :', len(images))

#load the model and processor
device = torch.device("cpu")
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-small')
model = AutoModel.from_pretrained('facebook/dinov2-small').to(device)

#Create Faiss index using FlatL2 type with 384 dimensions as this
#is the number of dimensions of the features
index = faiss.IndexFlatL2(384)

import time
t0 = time.time()
for image_path in images:
    img = Image.open(image_path).convert('RGB')
    with torch.no_grad():
        inputs = processor(images=img, return_tensors="pt").to(device)
        outputs = model(**inputs)
    features = outputs.last_hidden_state
    add_vector_to_index( features.mean(dim=1), index)

print('Extraction done in :', time.time()-t0)

#Store the index locally
faiss.write_index(index,"vector.index")

Number of images : 480
Extraction done in : 206.97534608840942


In [4]:
images = []
for root, dirs, files in os.walk('../gallery/photos/'):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.heic'):
            images.append(root + '/' + file)

print('Number of images :', len(images))

Number of images : 480


In [5]:
#pickel the images list
import pickle
with open('images.pkl', 'wb') as f:
    pickle.dump(images, f)

In [4]:
import faiss
import numpy as np
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image

#input image
image = Image.open(r"C:\Users\garra\Desktop\img_2835.heic").convert('RGB')

#Load the model and processor
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-small')
model = AutoModel.from_pretrained('facebook/dinov2-small').to(device)

#Extract the features
with torch.no_grad():
    inputs = processor(images=image, return_tensors="pt").to(device)
    outputs = model(**inputs)

#Normalize the features before search
embeddings = outputs.last_hidden_state
embeddings = embeddings.mean(dim=1)
vector = embeddings.detach().cpu().numpy()
vector = np.float32(vector)
faiss.normalize_L2(vector)

#Read the index file and perform search of top-3 images
index = faiss.read_index("vector.index")
d,i = index.search(vector,3)
print('distances:', d, 'indexes:', i)

distances: [[0.         0.07601181 0.08240125]] indexes: [[44 48 47]]


In [5]:
top_image_paths = [images[i] for i in i[0]]  # top_indexes[0] contains the indexes of the top 3 images

# Display the images
for img_path in top_image_paths:
    img = Image.open(img_path)
    img.show()